In [2]:
# generic imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
# notebook settings
%config IPCompleter.greedy=True
%load_ext autoreload
%autoreload 2 
# precision and plot settings
num_precision = 3
np.set_printoptions(precision=num_precision, suppress=True)
pd.set_option('display.float_format', lambda x: f'{x:,.{num_precision}f}')
pd.set_option("display.precision", num_precision)
pd.set_option('display.max_columns', None)
plt.rcParams['font.size'] = 16
plt.rcParams['legend.fontsize'] = 'large'
plt.rcParams['figure.titlesize'] = 'medium'
plt.rcParams['lines.linewidth'] = 2

In [3]:
# setup dir and import helper functions
import sys, os
sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'src'))
import helper_funcs as my_funcs

In [4]:
# from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow import data
from tensorflow.keras import callbacks

In [5]:
# # inspired by: https://stackoverflow.com/questions/62166588/how-to-obtain-filenames-during-prediction-while-using-tf-keras-preprocessing-ima
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function

# import numpy as np

# from tensorflow.python.data.ops import dataset_ops
# from tensorflow.python.keras.layers.preprocessing import image_preprocessing
# from tensorflow.python.keras.preprocessing import dataset_utils
# from tensorflow.python.ops import image_ops
# from tensorflow.python.ops import io_ops
# from tensorflow.python.util.tf_export import keras_export

# WHITELIST_FORMATS = ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

# ## Tensorflow override method to return fname as list as well as dataset

# def image_dataset_from_directory(directory,
#                                  labels='inferred',
#                                  label_mode='int',
#                                  class_names=None,
#                                  color_mode='rgb',
#                                  batch_size=32,
#                                  image_size=(256, 256),
#                                  shuffle=True,
#                                  seed=None,
#                                  validation_split=None,
#                                  subset=None,
#                                  interpolation='bilinear',
#                                  follow_links=False):
  
#   if labels != 'inferred':
#     if not isinstance(labels, (list, tuple)):
#       raise ValueError(
#           '`labels` argument should be a list/tuple of integer labels, of '
#           'the same size as the number of image files in the target '
#           'directory. If you wish to infer the labels from the subdirectory '
#           'names in the target directory, pass `labels="inferred"`. '
#           'If you wish to get a dataset that only contains images '
#           '(no labels), pass `label_mode=None`.')
#     if class_names:
#       raise ValueError('You can only pass `class_names` if the labels are '
#                        'inferred from the subdirectory names in the target '
#                        'directory (`labels="inferred"`).')
#   if label_mode not in {'int', 'categorical', 'binary', None}:
#     raise ValueError(
#         '`label_mode` argument must be one of "int", "categorical", "binary", '
#         'or None. Received: %s' % (label_mode,))
#   if color_mode == 'rgb':
#     num_channels = 3
#   elif color_mode == 'rgba':
#     num_channels = 4
#   elif color_mode == 'grayscale':
#     num_channels = 1
#   else:
#     raise ValueError(
#         '`color_mode` must be one of {"rbg", "rgba", "grayscale"}. '
#         'Received: %s' % (color_mode,))
#   interpolation = image_preprocessing.get_interpolation(interpolation)
#   dataset_utils.check_validation_split_arg(
#       validation_split, subset, shuffle, seed)

#   if seed is None:
#     seed = np.random.randint(1e6)
#   image_paths, labels, class_names = dataset_utils.index_directory(
#       directory,
#       labels,
#       formats=WHITELIST_FORMATS,
#       class_names=class_names,
#       shuffle=shuffle,
#       seed=seed,
#       follow_links=follow_links)

#   if label_mode == 'binary' and len(class_names) != 2:
#     raise ValueError(
#         'When passing `label_mode="binary", there must exactly 2 classes. '
#         'Found the following classes: %s' % (class_names,))

#   image_paths, labels = dataset_utils.get_training_or_validation_split(
#       image_paths, labels, validation_split, subset)

#   dataset = paths_and_labels_to_dataset(
#       image_paths=image_paths,
#       image_size=image_size,
#       num_channels=num_channels,
#       labels=labels,
#       label_mode=label_mode,
#       num_classes=len(class_names),
#       interpolation=interpolation)
#   if shuffle:
#     # Shuffle locally at each iteration
#     dataset = dataset.shuffle(buffer_size=batch_size * 8, seed=seed)
#   dataset = dataset.batch(batch_size)
#   # Users may need to reference `class_names`.
#   dataset.class_names = class_names
#   return dataset, image_paths

# def paths_and_labels_to_dataset(image_paths,
#                                 image_size,
#                                 num_channels,
#                                 labels,
#                                 label_mode,
#                                 num_classes,
#                                 interpolation):
#   """Constructs a dataset of images and labels."""
#   # TODO(fchollet): consider making num_parallel_calls settable
#   path_ds = dataset_ops.Dataset.from_tensor_slices(image_paths)
#   img_ds = path_ds.map(
#       lambda x: path_to_image(x, image_size, num_channels, interpolation))
#   if label_mode:
#     label_ds = dataset_utils.labels_to_dataset(labels, label_mode, num_classes)
#     img_ds = dataset_ops.Dataset.zip((img_ds, label_ds))
#   return img_ds


# def path_to_image(path, image_size, num_channels, interpolation):
#   img = io_ops.read_file(path)
#   img = image_ops.decode_image(
#       img, channels=num_channels, expand_animations=False)
#   img = image_ops.resize_images_v2(img, image_size, method=interpolation)
#   img.set_shape((image_size[0], image_size[1], num_channels))
#   return img

In [6]:
# # from keras: https://github.com/tensorflow/tensorflow/blob/v2.3.0/tensorflow/python/keras/preprocessing/image_dataset.py#L34-L206
# """Keras image dataset loading utilities."""
# # pylint: disable=g-classes-have-attributes
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function

# import numpy as np

# from tensorflow.python.data.ops import dataset_ops
# from tensorflow.python.keras.layers.preprocessing import image_preprocessing
# from tensorflow.python.keras.preprocessing import dataset_utils
# from tensorflow.python.ops import image_ops
# from tensorflow.python.ops import io_ops
# from tensorflow.python.util.tf_export import keras_export


# WHITELIST_FORMATS = ('.bmp', '.gif', '.jpeg', '.jpg', '.png')


# @keras_export('keras.preprocessing.image_dataset_from_directory', v1=[])
# def image_dataset_from_directory(directory,
#                                  labels='inferred',
#                                  label_mode='int',
#                                  class_names=None,
#                                  color_mode='rgb',
#                                  batch_size=32,
#                                  image_size=(256, 256),
#                                  shuffle=True,
#                                  seed=None,
#                                  validation_split=None,
#                                  subset=None,
#                                  interpolation='bilinear',
#                                  follow_links=False):
#   """Generates a `tf.data.Dataset` from image files in a directory.
#   If your directory structure is:
#   ```
#   main_directory/
#   ...class_a/
#   ......a_image_1.jpg
#   ......a_image_2.jpg
#   ...class_b/
#   ......b_image_1.jpg
#   ......b_image_2.jpg
#   ```
#   Then calling `image_dataset_from_directory(main_directory, labels='inferred')`
#   will return a `tf.data.Dataset` that yields batches of images from
#   the subdirectories `class_a` and `class_b`, together with labels
#   0 and 1 (0 corresponding to `class_a` and 1 corresponding to `class_b`).
#   Supported image formats: jpeg, png, bmp, gif.
#   Animated gifs are truncated to the first frame.
#   Arguments:
#     directory: Directory where the data is located.
#         If `labels` is "inferred", it should contain
#         subdirectories, each containing images for a class.
#         Otherwise, the directory structure is ignored.
#     labels: Either "inferred"
#         (labels are generated from the directory structure),
#         or a list/tuple of integer labels of the same size as the number of
#         image files found in the directory. Labels should be sorted according
#         to the alphanumeric order of the image file paths
#         (obtained via `os.walk(directory)` in Python).
#     label_mode:
#         - 'int': means that the labels are encoded as integers
#             (e.g. for `sparse_categorical_crossentropy` loss).
#         - 'categorical' means that the labels are
#             encoded as a categorical vector
#             (e.g. for `categorical_crossentropy` loss).
#         - 'binary' means that the labels (there can be only 2)
#             are encoded as `float32` scalars with values 0 or 1
#             (e.g. for `binary_crossentropy`).
#         - None (no labels).
#     class_names: Only valid if "labels" is "inferred". This is the explict
#         list of class names (must match names of subdirectories). Used
#         to control the order of the classes
#         (otherwise alphanumerical order is used).
#     color_mode: One of "grayscale", "rgb", "rgba". Default: "rgb".
#         Whether the images will be converted to
#         have 1, 3, or 4 channels.
#     batch_size: Size of the batches of data. Default: 32.
#     image_size: Size to resize images to after they are read from disk.
#         Defaults to `(256, 256)`.
#         Since the pipeline processes batches of images that must all have
#         the same size, this must be provided.
#     shuffle: Whether to shuffle the data. Default: True.
#         If set to False, sorts the data in alphanumeric order.
#     seed: Optional random seed for shuffling and transformations.
#     validation_split: Optional float between 0 and 1,
#         fraction of data to reserve for validation.
#     subset: One of "training" or "validation".
#         Only used if `validation_split` is set.
#     interpolation: String, the interpolation method used when resizing images.
#       Defaults to `bilinear`. Supports `bilinear`, `nearest`, `bicubic`,
#       `area`, `lanczos3`, `lanczos5`, `gaussian`, `mitchellcubic`.
#     follow_links: Whether to visits subdirectories pointed to by symlinks.
#         Defaults to False.
#   Returns:
#     A `tf.data.Dataset` object.
#       - If `label_mode` is None, it yields `float32` tensors of shape
#         `(batch_size, image_size[0], image_size[1], num_channels)`,
#         encoding images (see below for rules regarding `num_channels`).
#       - Otherwise, it yields a tuple `(images, labels)`, where `images`
#         has shape `(batch_size, image_size[0], image_size[1], num_channels)`,
#         and `labels` follows the format described below.
#   Rules regarding labels format:
#     - if `label_mode` is `int`, the labels are an `int32` tensor of shape
#       `(batch_size,)`.
#     - if `label_mode` is `binary`, the labels are a `float32` tensor of
#       1s and 0s of shape `(batch_size, 1)`.
#     - if `label_mode` is `categorial`, the labels are a `float32` tensor
#       of shape `(batch_size, num_classes)`, representing a one-hot
#       encoding of the class index.
#   Rules regarding number of channels in the yielded images:
#     - if `color_mode` is `grayscale`,
#       there's 1 channel in the image tensors.
#     - if `color_mode` is `rgb`,
#       there are 3 channel in the image tensors.
#     - if `color_mode` is `rgba`,
#       there are 4 channel in the image tensors.
#   """
#   if labels != 'inferred':
#     if not isinstance(labels, (list, tuple)):
#       raise ValueError(
#           '`labels` argument should be a list/tuple of integer labels, of '
#           'the same size as the number of image files in the target '
#           'directory. If you wish to infer the labels from the subdirectory '
#           'names in the target directory, pass `labels="inferred"`. '
#           'If you wish to get a dataset that only contains images '
#           '(no labels), pass `label_mode=None`.')
#     if class_names:
#       raise ValueError('You can only pass `class_names` if the labels are '
#                        'inferred from the subdirectory names in the target '
#                        'directory (`labels="inferred"`).')
#   if label_mode not in {'int', 'categorical', 'binary', None}:
#     raise ValueError(
#         '`label_mode` argument must be one of "int", "categorical", "binary", '
#         'or None. Received: %s' % (label_mode,))
#   if color_mode == 'rgb':
#     num_channels = 3
#   elif color_mode == 'rgba':
#     num_channels = 4
#   elif color_mode == 'grayscale':
#     num_channels = 1
#   else:
#     raise ValueError(
#         '`color_mode` must be one of {"rbg", "rgba", "grayscale"}. '
#         'Received: %s' % (color_mode,))
#   interpolation = image_preprocessing.get_interpolation(interpolation)
#   dataset_utils.check_validation_split_arg(
#       validation_split, subset, shuffle, seed)

#   if seed is None:
#     seed = np.random.randint(1e6)
#   image_paths, labels, class_names = dataset_utils.index_directory(
#       directory,
#       labels,
#       formats=WHITELIST_FORMATS,
#       class_names=class_names,
#       shuffle=shuffle,
#       seed=seed,
#       follow_links=follow_links)

#   if label_mode == 'binary' and len(class_names) != 2:
#     raise ValueError(
#         'When passing `label_mode="binary", there must exactly 2 classes. '
#         'Found the following classes: %s' % (class_names,))

#   image_paths, labels = dataset_utils.get_training_or_validation_split(
#       image_paths, labels, validation_split, subset)

#   dataset = paths_and_labels_to_dataset(
#       image_paths=image_paths,
#       image_size=image_size,
#       num_channels=num_channels,
#       labels=labels,
#       label_mode=label_mode,
#       num_classes=len(class_names),
#       interpolation=interpolation)
#   if shuffle:
#     # Shuffle locally at each iteration
#     dataset = dataset.shuffle(buffer_size=batch_size * 8, seed=seed)
#   dataset = dataset.batch(batch_size)
#   # Users may need to reference `class_names`.
#   dataset.class_names = class_names
#   return dataset


# def paths_and_labels_to_dataset(image_paths,
#                                 image_size,
#                                 num_channels,
#                                 labels,
#                                 label_mode,
#                                 num_classes,
#                                 interpolation):
#   """Constructs a dataset of images and labels."""
#   # TODO(fchollet): consider making num_parallel_calls settable
#   path_ds = dataset_ops.Dataset.from_tensor_slices(image_paths)
#   img_ds = path_ds.map(
#       lambda x: path_to_image(x, image_size, num_channels, interpolation))
#   if label_mode:
#     label_ds = dataset_utils.labels_to_dataset(labels, label_mode, num_classes)
#     img_ds = dataset_ops.Dataset.zip((img_ds, label_ds))
#   return img_ds


# def path_to_image(path, image_size, num_channels, interpolation):
#   img = io_ops.read_file(path)
#   img = image_ops.decode_image(
#       img, channels=num_channels, expand_animations=False)
#   img = image_ops.resize_images_v2(img, image_size, method=interpolation)
#   img.set_shape((image_size[0], image_size[1], num_channels))
#   return img

In [7]:
# my attempt

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.python.data.ops import dataset_ops
from tensorflow.python.keras.layers.preprocessing import image_preprocessing
from tensorflow.python.keras.preprocessing import dataset_utils
from tensorflow.python.ops import image_ops
from tensorflow.python.ops import io_ops
from tensorflow.python.util.tf_export import keras_export


WHITELIST_FORMATS = ('.bmp', '.gif', '.jpeg', '.jpg', '.png')


# @keras_export('keras.preprocessing.image_dataset_from_directory', v1=[])
def image_dataset_from_directory(directory,
                                 labels='inferred',
                                 label_mode='int',
                                 class_names=None,
                                 color_mode='rgb',
                                 batch_size=32,
                                 image_size=(256, 256),
                                 shuffle=True,
                                 seed=None,
                                 validation_split=None,
                                 subset=None,
                                 interpolation='bilinear',
                                 follow_links=False):
  """Generates a `tf.data.Dataset` from image files in a directory.
  If your directory structure is:
  ```
  main_directory/
  ...class_a/
  ......a_image_1.jpg
  ......a_image_2.jpg
  ...class_b/
  ......b_image_1.jpg
  ......b_image_2.jpg
  ```
  Then calling `image_dataset_from_directory(main_directory, labels='inferred')`
  will return a `tf.data.Dataset` that yields batches of images from
  the subdirectories `class_a` and `class_b`, together with labels
  0 and 1 (0 corresponding to `class_a` and 1 corresponding to `class_b`).
  Supported image formats: jpeg, png, bmp, gif.
  Animated gifs are truncated to the first frame.
  Arguments:
    directory: Directory where the data is located.
        If `labels` is "inferred", it should contain
        subdirectories, each containing images for a class.
        Otherwise, the directory structure is ignored.
    labels: Either "inferred"
        (labels are generated from the directory structure),
        or a list/tuple of integer labels of the same size as the number of
        image files found in the directory. Labels should be sorted according
        to the alphanumeric order of the image file paths
        (obtained via `os.walk(directory)` in Python).
    label_mode:
        - 'int': means that the labels are encoded as integers
            (e.g. for `sparse_categorical_crossentropy` loss).
        - 'categorical' means that the labels are
            encoded as a categorical vector
            (e.g. for `categorical_crossentropy` loss).
        - 'binary' means that the labels (there can be only 2)
            are encoded as `float32` scalars with values 0 or 1
            (e.g. for `binary_crossentropy`).
        - None (no labels).
    class_names: Only valid if "labels" is "inferred". This is the explict
        list of class names (must match names of subdirectories). Used
        to control the order of the classes
        (otherwise alphanumerical order is used).
    color_mode: One of "grayscale", "rgb", "rgba". Default: "rgb".
        Whether the images will be converted to
        have 1, 3, or 4 channels.
    batch_size: Size of the batches of data. Default: 32.
    image_size: Size to resize images to after they are read from disk.
        Defaults to `(256, 256)`.
        Since the pipeline processes batches of images that must all have
        the same size, this must be provided.
    shuffle: Whether to shuffle the data. Default: True.
        If set to False, sorts the data in alphanumeric order.
    seed: Optional random seed for shuffling and transformations.
    validation_split: Optional float between 0 and 1,
        fraction of data to reserve for validation.
    subset: One of "training" or "validation".
        Only used if `validation_split` is set.
    interpolation: String, the interpolation method used when resizing images.
      Defaults to `bilinear`. Supports `bilinear`, `nearest`, `bicubic`,
      `area`, `lanczos3`, `lanczos5`, `gaussian`, `mitchellcubic`.
    follow_links: Whether to visits subdirectories pointed to by symlinks.
        Defaults to False.
  Returns:
    A `tf.data.Dataset` object.
      - If `label_mode` is None, it yields `float32` tensors of shape
        `(batch_size, image_size[0], image_size[1], num_channels)`,
        encoding images (see below for rules regarding `num_channels`).
      - Otherwise, it yields a tuple `(images, labels)`, where `images`
        has shape `(batch_size, image_size[0], image_size[1], num_channels)`,
        and `labels` follows the format described below.
  Rules regarding labels format:
    - if `label_mode` is `int`, the labels are an `int32` tensor of shape
      `(batch_size,)`.
    - if `label_mode` is `binary`, the labels are a `float32` tensor of
      1s and 0s of shape `(batch_size, 1)`.
    - if `label_mode` is `categorial`, the labels are a `float32` tensor
      of shape `(batch_size, num_classes)`, representing a one-hot
      encoding of the class index.
  Rules regarding number of channels in the yielded images:
    - if `color_mode` is `grayscale`,
      there's 1 channel in the image tensors.
    - if `color_mode` is `rgb`,
      there are 3 channel in the image tensors.
    - if `color_mode` is `rgba`,
      there are 4 channel in the image tensors.
  """
  if labels != 'inferred':
    if not isinstance(labels, (list, tuple)):
      raise ValueError(
          '`labels` argument should be a list/tuple of integer labels, of '
          'the same size as the number of image files in the target '
          'directory. If you wish to infer the labels from the subdirectory '
          'names in the target directory, pass `labels="inferred"`. '
          'If you wish to get a dataset that only contains images '
          '(no labels), pass `label_mode=None`.')
    if class_names:
      raise ValueError('You can only pass `class_names` if the labels are '
                       'inferred from the subdirectory names in the target '
                       'directory (`labels="inferred"`).')
  if label_mode not in {'int', 'categorical', 'binary', None}:
    raise ValueError(
        '`label_mode` argument must be one of "int", "categorical", "binary", '
        'or None. Received: %s' % (label_mode,))
  if color_mode == 'rgb':
    num_channels = 3
  elif color_mode == 'rgba':
    num_channels = 4
  elif color_mode == 'grayscale':
    num_channels = 1
  else:
    raise ValueError(
        '`color_mode` must be one of {"rbg", "rgba", "grayscale"}. '
        'Received: %s' % (color_mode,))
  interpolation = image_preprocessing.get_interpolation(interpolation)
  dataset_utils.check_validation_split_arg(
      validation_split, subset, shuffle, seed)

  if seed is None:
    seed = np.random.randint(1e6)
  image_paths, labels, class_names = dataset_utils.index_directory(
      directory,
      labels,
      formats=WHITELIST_FORMATS,
      class_names=class_names,
      shuffle=shuffle,
      seed=seed,
      follow_links=follow_links)

  if label_mode == 'binary' and len(class_names) != 2:
    raise ValueError(
        'When passing `label_mode="binary", there must exactly 2 classes. '
        'Found the following classes: %s' % (class_names,))

  image_paths, labels = dataset_utils.get_training_or_validation_split(
      image_paths, labels, validation_split, subset)

  dataset = paths_and_labels_to_dataset(
      image_paths=image_paths,
      image_size=image_size,
      num_channels=num_channels,
      labels=labels,
      label_mode=label_mode,
      num_classes=len(class_names),
      interpolation=interpolation)
  if shuffle:
    # Shuffle locally at each iteration
    dataset = dataset.shuffle(buffer_size=batch_size * 8, seed=seed)
  dataset = dataset.batch(batch_size)
  # Users may need to reference `class_names`.
  dataset.class_names = class_names
####### WHAT I CHANGED ################
  return dataset, image_paths
####### WHAT I CHANGED ################

def paths_and_labels_to_dataset(image_paths,
                                image_size,
                                num_channels,
                                labels,
                                label_mode,
                                num_classes,
                                interpolation):
  """Constructs a dataset of images and labels."""
  # TODO(fchollet): consider making num_parallel_calls settable
  path_ds = dataset_ops.Dataset.from_tensor_slices(image_paths)
  img_ds = path_ds.map(
      lambda x: path_to_image(x, image_size, num_channels, interpolation))
  if label_mode:
    label_ds = dataset_utils.labels_to_dataset(labels, label_mode, num_classes)
    img_ds = dataset_ops.Dataset.zip((img_ds, label_ds))
  return img_ds


def path_to_image(path, image_size, num_channels, interpolation):
  img = io_ops.read_file(path)
  img = image_ops.decode_image(
      img, channels=num_channels, expand_animations=False)
  img = image_ops.resize_images_v2(img, image_size, method=interpolation)
  img.set_shape((image_size[0], image_size[1], num_channels))
  return img

In [8]:
directory = '../data/symlink_data/wild_est_test'

In [9]:
batch_size = 32
img_size = 256

In [22]:
# make a dataset of 2
X_train, X_train_img_paths = image_dataset_from_directory(
    directory, labels='inferred', class_names=None, 
    color_mode='rgb', batch_size=8, image_size=(img_size, img_size), shuffle=True, seed=42,
    validation_split=0.25, subset='training', interpolation='bilinear', follow_links=True
)

Found 8 files belonging to 2 classes.
Using 6 files for training.


In [11]:
img_paths

['../data/symlink_data/wild_est_test/Established Campground/satimg_ID_264_Established Campground_17_43.460267_-113.560938_rot180.png',
 '../data/symlink_data/wild_est_test/Wild Camping/satimg_ID_11390_Wild Camping_17_47.702711_-117.031512_rot270.png',
 '../data/symlink_data/wild_est_test/Established Campground/satimg_AZ_18_Established Campground_17_31.703691999999997_-111.063476.png',
 '../data/symlink_data/wild_est_test/Wild Camping/satimg_MT_7384_Wild Camping_17_48.782051_-114.283557_rot270.png',
 '../data/symlink_data/wild_est_test/Established Campground/satimg_ID_983_Established Campground_17_42.956692_-115.307018_rot270.png',
 '../data/symlink_data/wild_est_test/Wild Camping/satimg_AZ_17_Wild Camping_17_32.784379_-109.602116.png']

In [13]:
X_test, X_test_img_paths = image_dataset_from_directory(
    directory, labels='inferred', class_names=None, 
    color_mode='rgb', batch_size=batch_size, image_size=(img_size, img_size), shuffle=True, seed=42, 
    validation_split=0.25, subset='validation', interpolation='bilinear', follow_links=True
)

Found 8 files belonging to 2 classes.
Using 2 files for validation.


In [14]:
X_test_img_paths

['../data/symlink_data/wild_est_test/Established Campground/satimg_MT_1237_Established Campground_17_46.65782_-111.70844_rot180.png',
 '../data/symlink_data/wild_est_test/Wild Camping/satimg_MT_5076_Wild Camping_17_47.812919_-113.854707_rot180.png']

In [20]:
def get_class_weights(X_train):
    class_names = X_train.class_names
    labels = np.concatenate([y for x, y in X_train], axis=0)
    if np.ndim(labels) == 1:
        # for binary
        weights = [len(labels) - labels.sum(), labels.sum()]
    else:
        weights = list(np.sum(labels, axis=0))
    class_weights = {}
    # for class_, weight in zip(class_names, weights):
    #     class_weights[class_] = weight
    for i, weight in enumerate(weights):
        class_weights[i] = weight
    return class_names, class_weights

In [23]:
class_names, class_weights = get_class_weights(X_train)

In [24]:
class_weights

{0: 3, 1: 3}

In [15]:
num_classes = 2
epochs = 1 
AUTOTUNE = data.experimental.AUTOTUNE
img_height = 256
img_width = 256
nb_filters = 32    # number of convolutional filters to use - want 1 for each "feature" you think exists in images if more, more parameters
pool_size = (2, 2)  # pooling decreases image size, reduces computation, adds translational invariance
kernel_size = (2, 2)  # convolutional kernel size, slides over image to learn features
X_train = X_train.cache().shuffle(32).prefetch(buffer_size=AUTOTUNE) 
X_test = X_test.cache().prefetch(buffer_size=AUTOTUNE)

In [16]:
model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.Conv2D(nb_filters, (kernel_size[0], kernel_size[1]), padding='same', activation='relu'), # was 16, 32, 64
    layers.MaxPooling2D(pool_size=pool_size),
    layers.Conv2D(nb_filters*2, (kernel_size[0], kernel_size[1]), padding='same', activation='relu'), # drop layers.. for initial testing
    layers.MaxPooling2D(pool_size=pool_size),
    layers.Conv2D(nb_filters*3, (kernel_size[0], kernel_size[1]), padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=pool_size),
    layers.Conv2D(nb_filters*4, (kernel_size[0], kernel_size[1]), padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=pool_size),
    layers.Flatten(),
    layers.Dense(256, activation='relu'), # increase this? add another dense layer?
    layers.Dropout(0.5),
    #layers.Dense(num_classes, activation='relu') # or sigmoid for binary? relu?
    layers.Dense(1, activation='sigmoid') # or sigmoid for binary? relu?
])

In [17]:
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 256, 256, 3)       0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 256, 256, 3)       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 256, 256, 3)       0         
_________________________________________________________________
random_zoom (RandomZoom)     (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      8

In [19]:
# run it
history = model.fit(
            X_train,
            validation_data = X_test,
            epochs = 5,
            #batch_size=batch_size,
            verbose = 1,
#             callbacks=my_callbacks
) # weighted_metrics

Epoch 1/5
1/1 [==============================] - 0s 233ms/step - loss: 0.7025 - accuracy: 0.3333 - val_loss: 0.7181 - val_accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - 0s 59ms/step - loss: 0.7819 - accuracy: 0.5000 - val_loss: 0.6475 - val_accuracy: 0.5000
Epoch 3/5
1/1 [==============================] - 0s 56ms/step - loss: 0.6793 - accuracy: 0.5000 - val_loss: 0.6652 - val_accuracy: 0.5000
Epoch 4/5
1/1 [==============================] - 0s 59ms/step - loss: 0.6546 - accuracy: 0.6667 - val_loss: 0.6750 - val_accuracy: 0.5000
Epoch 5/5
1/1 [==============================] - 0s 62ms/step - loss: 0.5894 - accuracy: 0.6667 - val_loss: 0.6842 - val_accuracy: 0.5000


In [ ]:
# testing this plot function
model_name = 'test'
fig, axs = plt.subplots(1, 2, figsize=(10, 8))
my_funcs.plot_train_val_acc(history, 5, model_name, axs)